# Read TileWall data and convert it to multple .json files


*I took the data from the google doc on the SVL drive, and converted it first to csv files.  Here I'm reading this in to create individual json files for each line.  I will write another code that will compile each individual json file into one json file to be read into the app.  I think this may be the easiest method for others to add additional objects.*

In [1]:
import pandas as pd
import numpy as np
import os
import wikipedia
import wptools #https://github.com/siznax/wptools

###  A function to grab an image from Wikipedia

*https://wikipedia.readthedocs.io/en/latest/quickstart.html*

In [5]:
def getWikiImages(nme, category):
    nme = nme.replace('(',' ')
    nme = nme.replace(')',' ')
    if (nme[0] == "M" and nme[1].isdigit()):
        nme = '"Messier '+nme[1:]+'"'
    if (category == "Galaxies"):
        category = "Galaxy"
    if (category == "Galaxy Clusters"):
        category = "Galaxy Cluster"
    if (category == "Nebulae"):
        category = "Nebula"
    if (category == "Star Clusters"):
        category = "Star Cluster"
    if (category == "Stars"):
        category = "Star"
    if (category == "Constellations"):
        category = "Constellation"
    search = nme + " " + '"'+category+'"' + " astronomy"
    site = wikipedia.search(search)
    print(search)
    if (site):
        print("wiki page: ", site[0])
        page = None
        try:
            page = wikipedia.page(site[0])
        except wikipedia.exceptions.DisambiguationError as e:
            #site = e.options
            pass
        if (page):
            img = []
            for i in page.images:
                if (("jpg" in i or "png" in i) and "Celestia" not in i):
                    img.append(i)
                    
                    #get more info with wptools
            #https://github.com/siznax/wptools/wiki/
            wpage = wptools.page(site[0])
            wpage.get_query()

            #get the main image
            mainImg = None
            if (wpage.images()):
                for i in wpage.images():
                    if (i['kind'] == 'query-pageimage'):
                        mainImg = i['url']

                if (mainImg):
                    img2 = img
                    img = [mainImg]
                    for i in img2:
                        if (i != mainImg):
                            img.append(i)
                        
            return img, page.url

    return None, None

In [6]:
files = ['TileWallGalaxies.csv', 'TileWallGalaxyClusters.csv','TileWallNebulae.csv', \
         'TileWallStarClusters.csv','TileWallStars.csv',  'TileWallConstellations.csv' ]
tpe = ['Galaxies', 'Galaxy Clusters', 'Nebulae', 'Star Clusters', 'Stars', 'Constellations']
#not sure what to do about the 'Constellation Detailed Notes'...

In [7]:
for i,f in enumerate(files):
    df = pd.read_csv(f, encoding = "ISO-8859-1")
    for index, row in df.iterrows():
        if (isinstance(row['Object'], str)):
            row['Category'] = tpe[i]
            d = {}
            nme = row['Object']
            fname = os.path.join('objectFiles',''.join(e for e in nme if e.isalnum())+'.json')
            print(fname)
            
            d[nme] = row
            d[nme].pop('Object')
            imgurls, url = getWikiImages(nme, tpe[i])
            d[nme]['images'] = None
            d[nme]['wikipedia'] = None
            if (imgurls):
                d[nme]['images'] = imgurls
                d[nme]['wikipedia'] = url
                print(imgurls[0])
            pd.DataFrame(d).to_json(fname)
            print("")

objectFiles/AndromedaM31.json
Andromeda M31  "Galaxy" astronomy
wiki page:  Andromeda Galaxy


en.wikipedia.org (query) Andromeda Galaxy
en.wikipedia.org (query) Andromeda Galaxy (&plcontinue=74331|0|NGC_147)
en.wikipedia.org (query) Andromeda Galaxy (&plcontinue=74331|0|Pa...
en.wikipedia.org (imageinfo) File:Andromeda Galaxy (with h-alpha).jpg
Andromeda Galaxy (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(4)> Andromeda, M 31, Messier 31, NGC 224
  assessments: <dict(1)> Astronomy
  description: spiral galaxy within the Local Group
  extext: <str(1336)> The **Andromeda Galaxy** ( ), also known as ...
  extract: <str(1496)> <p class="mw-empty-elt"></p><p>The <b>Andro...
  image: <list(6)> {'kind': 'query-pageimage', 'file': 'File:Andro...
  label: Andromeda Galaxy
  length: 95,286
  links: <list(1128)> 10 Andromedae, 1000000000000 (number), 11 An...
  modified: <dict(1)> page
  pageid: 74331
  random: Luca Pompilio
  redirects: <list(29)> {'pageid': 250873, 'ns': 0, 'title': 'NGC ...
  requests: <list(4)> query, query, query, imageinfo
  title: Andromeda Galaxy
  u

https://upload.wikimedia.org/wikipedia/commons/9/98/Andromeda_Galaxy_%28with_h-alpha%29.jpg

objectFiles/AntennaeGalaxies.json
Antennae Galaxies "Galaxy" astronomy
wiki page:  Antennae Galaxies


en.wikipedia.org (query) Antennae Galaxies
en.wikipedia.org (query) Antennae Galaxies (&plcontinue=1111758|0...
en.wikipedia.org (imageinfo) File:Antennae Galaxies reloaded.jpg
Antennae Galaxies (en) data
{
  WARNINGS: <dict(1)> extracts
  assessments: <dict(1)> Astronomy
  description: interacting galaxies in the constellation Corvus
  extext: <str(344)> The **Antennae Galaxies** , also known as **N...
  extract: <str(360)> <p>The <b>Antennae Galaxies</b>, also known ...
  image: <list(4)> {'kind': 'query-pageimage', 'file': 'File:Anten...
  label: Antennae Galaxies
  length: 9,153
  links: <list(665)> 31 Crateris, 47 Tucanae, Andromeda Galaxy, An...
  modified: <dict(1)> page
  pageid: 1111758
  random: Padraig Clancy
  redirects: <list(18)> {'pageid': 1556519, 'ns': 0, 'title': 'The...
  requests: <list(3)> query, query, imageinfo
  title: Antennae Galaxies
  url: https://en.wikipedia.org/wiki/Antennae_Galaxies
  url_raw: https://en.wikipedia.org/wiki/Antennae_Galaxies?action=raw
  

https://upload.wikimedia.org/wikipedia/commons/3/30/Antennae_Galaxies_reloaded.jpg

objectFiles/BlackEyeGalaxyM64.json
Black Eye Galaxy M64  "Galaxy" astronomy
wiki page:  Black Eye Galaxy


...

en.wikipedia.org (query) Ursa Minor
en.wikipedia.org (imageinfo) File:Ursa Minor IAU.svg
Ursa Minor (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(4)> Little Bear, Little Dipper, UMi, Ursae Minoris
  assessments: <dict(1)> Astronomy
  description: <str(90)> constellation in the northern celestial h...
  extext: <str(1294)> **Ursa Minor** (Latin: "Lesser Bear", contra...
  extract: <str(1360)> <p class="mw-empty-elt"></p><p><b>Ursa Mino...
  image: <list(2)> {'kind': 'query-pageimage', 'file': 'File:Ursa ...
  label: Ursa Minor
  length: 49,782
  links: <list(353)> 10 Ursae Minoris, 10th parallel south, 11 Urs...
  modified: <dict(1)> page
  pageid: 31773
  random: Supernode
  redirects: <list(7)> {'pageid': 17705, 'ns': 0, 'title': 'Little...
  requests: <list(2)> query, imageinfo
  title: Ursa Minor
  url: https://en.wikipedia.org/wiki/Ursa_Minor
  url_raw: https://en.wikipedia.org/wiki/Ursa_Minor?action=raw
  watchers: 120
  wikibase: Q10478
  wikidata_url: https://www.w

https://upload.wikimedia.org/wikipedia/commons/c/cb/Ursa_Minor_IAU.svg

objectFiles/Virgo.json
Virgo "Constellation" astronomy
wiki page:  NGC 4567 and NGC 4568


en.wikipedia.org (query) NGC 4567 and NGC 4568
en.wikipedia.org (query) NGC 4567 and NGC 4568 (&plcontinue=40868...
en.wikipedia.org (imageinfo) File:NGC4568.jpg


https://upload.wikimedia.org/wikipedia/commons/b/b5/NGC4568.jpg



NGC 4567 and NGC 4568 (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(4)> Siamese Twins, Butterfly Galaxies, KPG 347, VV 219
  assessments: <dict(1)> Astronomy
  description: galaxy pair in the constellation Virgo
  extext: <str(713)> **NGC 4567 and NGC 4568** (nicknamed the _**S...
  extract: <str(762)> <p><b>NGC 4567 and NGC 4568</b> (nicknamed t...
  image: <list(4)> {'kind': 'query-pageimage', 'file': 'File:NGC45...
  label: NGC 4567 and NGC 4568
  length: 3,073
  links: <list(543)> Absolute magnitude, Angular diameter, Antenna...
  modified: <dict(1)> page
  pageid: 4086864
  random: Yo No Sé Mañana
  redirects: <list(10)> {'pageid': 4617960, 'ns': 0, 'title': 'NGC...
  requests: <list(3)> query, query, imageinfo
  title: NGC 4567 and NGC 4568
  url: https://en.wikipedia.org/wiki/NGC_4567_and_NGC_4568
  url_raw: https://en.wikipedia.org/wiki/NGC_4567_and_NGC_4568?act...
  wikibase: Q869408
  wikidata_url: https://www.wikidata.org/wiki/Q869408
}
